# Getting started with PyEpiDoc

## Loading an EpiDoc file and accessing text

In [1]:
# Load the API
from pyepidoc import EpiDoc, EpiDocCorpus
from pyepidoc.shared.display import show_list

In [2]:
# Load an EpiDoc file without validation
doc = EpiDoc('examples/ISic000001_tokenized.xml')

In [3]:
# Load an EpiDoc file and validate
doc2 = EpiDoc('examples/ISic000001_tokenized.xml', validate_on_load=True)

RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile not

examples/ISic000001_tokenized.xml is a valid EpiDoc file


Allowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to com

In [4]:
# Print validation result
print(doc2.validation_result)

examples/ISic000001_tokenized.xml is valid


In [5]:
# Print normalized tokens as a string
print(doc.text_normalized)

Dis manibus Zethi vixit annis VI


In [6]:
# Print Leiden tokens as a string
print(doc.text_leiden)


Dis · man(ibus)
 · Zethi
vix(it) · a(nnis) · VI


In [7]:
# Print the translation text
print(doc.translation_text)

## Load a corpus

In [8]:
# Load the corpus
corpus = EpiDocCorpus('example_corpus')

In [9]:
# Count the documents
print(corpus.doc_count)

70


In [11]:
# Get the ids of the documents in the corpus
print(show_list(corpus.ids))


- ISic000001
- ISic000002
- ISic000003
- ISic000004
- ISic000005
- ISic000006
- ISic000007
- ISic000008
- ISic000009
- ISic000010
- ISic000800
- ISic000801
- ISic000802
- ISic000803
- ISic000804
- ISic000805
- ISic000806
- ISic000807
- ISic000809
- ISic000810
- ISic000811
- ISic000812
- ISic000813
- ISic000814
- ISic000815
- ISic000816
- ISic000817
- ISic000818
- ISic000819
- ISic000820
- ISic000821
- ISic000822
- ISic000823
- ISic000824
- ISic000825
- ISic000826
- ISic000827
- ISic000828
- ISic000829
- ISic000830
- ISic000831
- ISic000832
- ISic000833
- ISic000834
- ISic000835
- ISic000836
- ISic000837
- ISic000838
- ISic000839
- ISic000840
- ISic000841
- ISic000842
- ISic000843
- ISic000844
- ISic000845
- ISic000846
- ISic000847
- ISic000848
- ISic000849
- ISic000850
- ISic000851
- ISic000852
- ISic000853
- ISic000854
- ISic000855
- ISic000856
- ISic000857
- ISic000858
- ISic000859
- ISic000860


## Filter the corpus

In [ ]:
# Find all the funerary 